# LLM Zoomcamp 2024 - Session #3 - Homework

Author: José Victor

## Q1. Getting the embeddings model

First, we will get the embeddings models `multi-qa-distilbert-cos-v1` from [the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* ( ) -0.24
* ( ) -0.04
* (X) 0.07
* ( ) 0.27

In [1]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)
user_question = "I just discovered the course. Can I still join it?"
embeddings = embedding_model.encode(user_question)
print(embeddings[0])

0.078222655


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions for `"machine-learning-zoomcamp"`. After filtering, you should have only 375 documents.

## Q2. Create the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

* Create a list `embeddings`
* Iterate over each document
* `qa_text = f'{question} {text}'`
* compute the embedding for `qa_text`, append to `embeddings`
* At the end, let `X = np.array(embeddings)` (`import numpy as np`)

What's the shape of X? (`X.shape`). Include the parantheses.

Answer: `(948, 768)`

In [2]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
embeddings = []

for doc in documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

In [4]:
import numpy as np

X = np.array(embeddings)
print(X.shape)

(948, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return 1.0). This means that in order to compute the cosine similarity, it's sufficient to multiply the matrix `X` by the vector `v`:

```python
scores = X.dot(v)
```

What's the highest score in the results?

* (X) 65.0
* ( ) 6.5
* ( ) 0.65
* ( ) 0.065

In [5]:
v = embedding_model.encode(user_question)
scores = X.dot(v)
print(scores.max())

0.6506573


## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGPT or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hit-rate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```
Now use the code from the module to calculate the hit-rate of `VectorSearchEngine` with `num_results=5`.

What did you get?

* ( ) 0.93
* ( ) 0.73
* ( ) 0.53
* ( ) 0.33

In [15]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [18]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's performm the search of the same query from Q1.

What's the ID of the document with the highest score?

Answer: `rUjmn5ABn2SjyfgHkQ7g`

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [8]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
for idx, doc in enumerate(documents):
    doc["text_vector"] = embeddings[idx]
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [11]:
user_question = "I just discovered the course. Can I still join it?"
vector_search_user_question = embedding_model.encode(user_question)

In [27]:
query = {"field": "text_vector",
        "query_vector": vector_search_user_question,
        "k": X.shape[0],
        "num_candidates": 10000}

In [28]:
res = es_client.search(index=index_name, knn=query, source=["question"])

In [29]:
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'rUjmn5ABn2SjyfgHkQ7g',
  '_score': 0.82532895,
  '_source': {'question': 'The course has already started. Can I still join it?'}},
 {'_index': 'course-questions',
  '_id': '7kjmn5ABn2SjyfgHZwyY',
  '_score': 0.78731877,
  '_source': {'question': 'Course - Can I still join the course after the start date?'}},
 {'_index': 'course-questions',
  '_id': 'Dkjmn5ABn2SjyfgHag3E',
  '_score': 0.7583767,
  '_source': {'question': 'How can we contribute to the course?'}},
 {'_index': 'course-questions',
  '_id': '80jmn5ABn2SjyfgHaAwu',
  '_score': 0.7583207,
  '_source': {'question': 'Course - Can I follow the course after it finishes?'}},
 {'_index': 'course-questions',
  '_id': '7Ejmn5ABn2SjyfgHZgy4',
  '_score': 0.75614965,
  '_source': {'question': 'Course - When will the course start?'}},
 {'_index': 'course-questions',
  '_id': 'sEjmn5ABn2SjyfgHkg4u',
  '_score': 0.73585373,
  '_source': {'question': 'I just joined. What should I do next? How can I 

In [31]:
res["hits"]["hits"][0]["_id"]

'rUjmn5ABn2SjyfgHkQ7g'

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hit-rate for our dataset for Elastic?

* ( ) 0.93
* ( ) 0.73
* ( ) 0.53
* ( ) 0.33